In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import re
import nltk
import string
from collections import Counter
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import ParameterGrid
from sklearn.svm import SVC
from multiprocessing import Pool
from transformers import BertTokenizer
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nlp-getting-started/sample_submission.csv
/kaggle/input/nlp-getting-started/train.csv
/kaggle/input/nlp-getting-started/test.csv


In [2]:
sample_submission = pd.read_csv("/kaggle/input/nlp-getting-started/sample_submission.csv")
train_df = pd.read_csv("/kaggle/input/nlp-getting-started/train.csv")

train_df, val_df = train_test_split(train_df, test_size=0.2, random_state=42)

test_df = pd.read_csv("/kaggle/input/nlp-getting-started/test.csv")

print("*************** SAMPLE SUBMISSION *****************")
display(sample_submission)

print("\n*************** TRAIN DATA *****************")
display(train_df)

print("\n*************** VAL DATA *****************")
display(val_df)

print("\n*************** TEST DATA *****************")
display(test_df)

*************** SAMPLE SUBMISSION *****************


,id,target
0,0,0
1,2,0
2,3,0
3,9,0
4,11,0
...,...,...
3258,10861,0
3259,10865,0
3260,10868,0
3261,10874,0



*************** TRAIN DATA *****************


,id,keyword,location,text,target
4996,7128,military,Texas,Courageous and honest analysis of need to use ...,1
3263,4688,engulfed,NaN,@ZachZaidman @670TheScore wld b a shame if tha...,0
4907,6984,massacre,Cottonwood Arizona,Tell @BarackObama to rescind medals of 'honor'...,1
2855,4103,drought,"Spokane, WA",Worried about how the CA drought might affect ...,1
4716,6706,lava,"Medan,Indonesia",@YoungHeroesID Lava Blast &amp; Power Red #Pan...,0
...,...,...,...,...,...
5226,7470,obliteration,Merica!,@Eganator2000 There aren't many Obliteration s...,0
5390,7691,panic,NaN,just had a panic attack bc I don't have enough...,0
860,1242,blood,NaN,Omron HEM-712C Automatic Blood Pressure Monito...,0
7603,10862,NaN,NaN,Officials say a quarantine is in place at an A...,1



*************** VAL DATA *****************


,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0
...,...,...,...,...,...
1835,2640,crashed,Somewhere,@SmusX16475 Skype just crashed u host,0
506,731,attacked,Arundel,Christian Attacked by Muslims at the Temple Mo...,1
3592,5131,fatal,"New South Wales, Australia",Man charged over fatal crash near Dubbo refuse...,1
6740,9657,thunderstorm,NaN,#usNWSgov Severe Weather Statement issued Augu...,1



*************** TEST DATA *****************


,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan
...,...,...,...,...
3258,10861,NaN,NaN,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...
3259,10865,NaN,NaN,Storm in RI worse than last hurricane. My city...
3260,10868,NaN,NaN,Green Line derailment in Chicago http://t.co/U...
3261,10874,NaN,NaN,MEG issues Hazardous Weather Outlook (HWO) htt...


In [3]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6090 entries, 4996 to 7270
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        6090 non-null   int64 
 1   keyword   6042 non-null   object
 2   location  4070 non-null   object
 3   text      6090 non-null   object
 4   target    6090 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 285.5+ KB


In [4]:
train_df.text

4996    Courageous and honest analysis of need to use ...
3263    @ZachZaidman @670TheScore wld b a shame if tha...
4907    Tell @BarackObama to rescind medals of 'honor'...
2855    Worried about how the CA drought might affect ...
4716    @YoungHeroesID Lava Blast &amp; Power Red #Pan...
                              ...                        
5226    @Eganator2000 There aren't many Obliteration s...
5390    just had a panic attack bc I don't have enough...
860     Omron HEM-712C Automatic Blood Pressure Monito...
7603    Officials say a quarantine is in place at an A...
7270    I moved to England five years ago today. What ...
Name: text, Length: 6090, dtype: object

In [5]:
train_texts = train_df.text.values.tolist()
train_classes = train_df.target.values.tolist()

val_texts = val_df.text.values.tolist()
val_classes = val_df.target.values.tolist()

test_texts = test_df.text.values.tolist()

In [6]:
def print_random_samples(texts, classes):
    np.random.seed(42)
    random_entries = np.random.randint(0, len(texts), 10)
    
    for i in random_entries:
        print(f"Entry (Class {classes[i]}) {i}: {texts[i]}\n")

In [7]:
print_random_samples(train_texts, train_classes)

Entry (Class 1) 860: If you were the NWS wth a rotating storm w/ a report of a 'HUGE' / 'MASSIVE' / 'VIOLENT' tornado what would you do? https://t.co/J3dI85IST5

Entry (Class 0) 5390: Demolished My Personal Best  http://t.co/ImULLBvUEd

Entry (Class 0) 5226: #sing #tsunami Beginners #computer tutorial.: http://t.co/ukQYbhxMQI Everyone Wants To Learn To Build A Pc. Re http://t.co/iDWS2ZgYsa

Entry (Class 0) 5191: Survival Kit Whistle Fire Starter Wire Saw Cree Torch Emergency Blanket S knife  - Full reÛ_ http://t.co/2OroYUNYM2 http://t.co/C9JnXz3DXC

Entry (Class 0) 3772: Buddha was 'man for his time' - massive urbanisation and social upheaval also challenged Brahmans dominance ('Genius of the Ancient World').

Entry (Class 1) 3092: Scary stuff in San Antonio. Stars head coach Dan Hughes was run into on the sidelines and fell over chairs onto the floor. Stretcher now out

Entry (Class 0) 5734: Free Ebay Sniping RT? http://t.co/B231Ul1O1K Lumbar Extender Back Stretcher Excellent Conditi

# Preprocessing

In [8]:
def apply(func, train_texts, validation_texts, test_texts):
    train_texts = [func(text) for text in train_texts]
    validation_texts = [func(text) for text in validation_texts]
    test_texts = [func(text) for text in test_texts]

    return train_texts, validation_texts, test_texts

# Lowercase the whole text
def lowercase(text):
    return text.lower()

# Remove punctuations
def remove_punctuation(text):
    return ''.join(character for character in text if character not in string.punctuation)


# Remove numbers
def remove_numbers(text):
    return re.sub(r'\d+', '', text)



train_texts, val_texts, test_texts = apply(lowercase, train_texts, val_texts, test_texts)
train_texts, val_texts, test_texts = apply(remove_punctuation, train_texts, val_texts, test_texts)
train_texts, val_texts, test_texts = apply(remove_numbers, train_texts, val_texts, test_texts)

print_random_samples(train_texts, train_classes)

Entry (Class 1) 860: if you were the nws wth a rotating storm w a report of a huge  massive  violent tornado what would you do httpstcojdiist

Entry (Class 0) 5390: demolished my personal best  httptcoimullbvued

Entry (Class 0) 5226: sing tsunami beginners computer tutorial httptcoukqybhxmqi everyone wants to learn to build a pc re httptcoidwszgysa

Entry (Class 0) 5191: survival kit whistle fire starter wire saw cree torch emergency blanket s knife   full reû httptcooroyunym httptcocjnxzdxc

Entry (Class 0) 3772: buddha was man for his time  massive urbanisation and social upheaval also challenged brahmans dominance genius of the ancient world

Entry (Class 1) 3092: scary stuff in san antonio stars head coach dan hughes was run into on the sidelines and fell over chairs onto the floor stretcher now out

Entry (Class 0) 5734: free ebay sniping rt httptcobulok lumbar extender back stretcher excellent condition please favorite amp share

Entry (Class 0) 466: brian shaw  jj hickson  ken

In [9]:
# Tokenize
def tokenize(text):
    return nltk.word_tokenize(text, language="english")


# Stopword filtering
def stopword_filtering(text_tokens):
    stop_words = nltk.corpus.stopwords.words('english')

    return [token for token in text_tokens if token not in stop_words]


# Stemming
def stemming(text_tokens):
    porter = nltk.stem.porter.PorterStemmer()
    return [porter.stem(token) for token in text_tokens]


train_texts, val_texts, test_texts = apply(tokenize, train_texts, val_texts, test_texts)
train_texts, val_texts, test_texts = apply(stopword_filtering, train_texts, val_texts, test_texts)
train_texts, val_texts, test_texts = apply(stemming, train_texts, val_texts, test_texts)

In [10]:
def stitch_text_tokens_together(text_tokens):
    return " ".join(text_tokens)


train_texts, val_texts, test_texts = apply(stitch_text_tokens_together, train_texts, val_texts,
                                                  test_texts)

print_random_samples(train_texts, train_classes)

Entry (Class 1) 860: nw wth rotat storm w report huge massiv violent tornado would httpstcojdiist

Entry (Class 0) 5390: demolish person best httptcoimullbvu

Entry (Class 0) 5226: sing tsunami beginn comput tutori httptcoukqybhxmqi everyon want learn build pc httptcoidwszgysa

Entry (Class 0) 5191: surviv kit whistl fire starter wire saw cree torch emerg blanket knife full reû httptcooroyunym httptcocjnxzdxc

Entry (Class 0) 3772: buddha man time massiv urbanis social upheav also challeng brahman domin geniu ancient world

Entry (Class 1) 3092: scari stuff san antonio star head coach dan hugh run sidelin fell chair onto floor stretcher

Entry (Class 0) 5734: free ebay snipe rt httptcobulok lumbar extend back stretcher excel condit pleas favorit amp share

Entry (Class 0) 466: brian shaw jj hickson kenneth fari tri defend lamarcu aldridg blood volcano httptcotwgpmmd

Entry (Class 0) 5334: downtown emerg servic center hire chemic depend counselor intern seattl appli job httptcoskqpwsno

In [11]:
tfidf_vectorizer = TfidfVectorizer(use_idf=True).fit(train_texts)

train_tfidf_vec = tfidf_vectorizer.transform(train_texts)
val_tfidf_vec = tfidf_vectorizer.transform(val_texts)
test_tfidf_vec = tfidf_vectorizer.transform(test_texts)

In [12]:
svd_tfidf = TruncatedSVD(n_components=400, n_iter=100, random_state=42).fit(train_tfidf_vec)

train_tfidf_vec = svd_tfidf.transform(train_tfidf_vec)
val_tfidf_vec = svd_tfidf.transform(val_tfidf_vec)
test_tfidf_vec = svd_tfidf.transform(test_tfidf_vec)

# Training

In [13]:
def evaluate_svc(params):
    model = SVC(**params['params']).fit(params['data']['train_x'], params['data']['train_y'])
    val_score = model.score(params['data']['val_x'], params['data']['val_y'])

    return {'val_acc': val_score, 'clf': model }

def evaluate_ada(params):
    model = AdaBoostClassifier(**params['params']).fit(params['data']['train_x'], params['data']['train_y'])
    val_score = model.score(params['data']['val_x'], params['data']['val_y'])

    return {'val_acc': val_score, 'clf': model }

def evaluate_knn(params):
    model = KNeighborsClassifier(**params['params']).fit(params['data']['train_x'], params['data']['train_y'])
    val_score = model.score(params['data']['val_x'], params['data']['val_y'])

    return {'val_acc': val_score, 'clf': model }

def hyperparameter_grid_search(evaluation_func, param_grid_list, train_x, train_y, val_x, val_y):
    param_grid = ParameterGrid(param_grid_list)

    multiprocess_params = list(map(lambda p: {
        'params': p,
        'data': {
            'train_x': train_x,
            'train_y': train_y,
            'val_x': val_x,
            'val_y': val_y
        }}, param_grid))

    with Pool(4) as p:
        results = p.map(evaluation_func, multiprocess_params)
        results.sort(key=lambda d: d['val_acc'], reverse=True)

        print("Best 3 classifiers: ")
        for r in results[:3]:
            print(f"Acc: {r['val_acc']:.3f}, {r['clf']}")

        return results[0]

In [14]:
dummy_clf = DummyClassifier(strategy="most_frequent").fit(train_tfidf_vec, train_classes)
dummy_clf.score(val_tfidf_vec, val_classes)

0.5738673670387393

In [15]:
best_svc = hyperparameter_grid_search(evaluate_svc, {
    "C": np.arange(0.7, 1.2, 0.05)
}, train_tfidf_vec, train_classes, val_tfidf_vec, val_classes)['clf']

Best 3 classifiers: 
Acc: 0.810, SVC(C=0.9000000000000001)
Acc: 0.809, SVC(C=0.9500000000000002)
Acc: 0.809, SVC(C=1.0000000000000002)


In [16]:
best_ada = hyperparameter_grid_search(evaluate_ada, {
    "n_estimators": [100,  200, 250, 300],
    "learning_rate": [0.2, 0.5, 0.7]
}, train_tfidf_vec, train_classes, val_tfidf_vec, val_classes)['clf']

Best 3 classifiers: 
Acc: 0.773, AdaBoostClassifier(learning_rate=0.2, n_estimators=300)
Acc: 0.773, AdaBoostClassifier(learning_rate=0.5, n_estimators=200)
Acc: 0.772, AdaBoostClassifier(learning_rate=0.2, n_estimators=250)


In [17]:
best_knn = hyperparameter_grid_search(evaluate_knn, {
    "n_neighbors": [1, 2, 3, 5, 6, 7, 8, 10, 50, 100, 200, 250, 300]
}, train_tfidf_vec, train_classes, val_tfidf_vec, val_classes)['clf']

Best 3 classifiers: 
Acc: 0.727, KNeighborsClassifier()
Acc: 0.726, KNeighborsClassifier(n_neighbors=8)
Acc: 0.724, KNeighborsClassifier(n_neighbors=10)


# Predict test

In [18]:
test_predictions = best_svc.predict(test_tfidf_vec)

sample_submission["target"] = test_predictions

display(sample_submission.head(20))

sample_submission.to_csv("submission.csv", index=False)

,id,target
0,0,1
1,2,1
2,3,1
3,9,1
4,11,1
5,12,1
6,21,0
7,22,0
8,27,0
9,29,0
